In [9]:
from glob import glob
from pathlib import Path
import os
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
#to access the files
all_file_path = glob('/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/*.edf')
print(len(all_file_path))

28


In [6]:
all_file_path[0]

'/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/s01.edf'

In [11]:
print(all_file_path)


['/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/s01.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/h01.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/h14.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/s14.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/s02.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/h02.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/h03.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/s03.edf', '/Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/s07.edf', '/Users/folasewaabdulsalam/Signal_Processing/

In [13]:


# Splitting the data file path into healthy and schizophrenia patients

healthy_file_path = [i for i in all_file_path if 'h' in i.split('/')[-1]]  # Extract filename using '/'
schizo_patient_file_path = [i for i in all_file_path if 's' in i.split('/')[-1]]  # Extract filename using '/'

print(len(healthy_file_path), len(schizo_patient_file_path))


14 14


In [14]:
#a function to read the path and get the data from it
def read_data(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True)
    data.set_eeg_reference() #set the channel references, this line of code will get set the reference as an average of the channels
    data.filter(l_freq = 0.5, h_freq = 45) #setting low frequency as 0.5 and high frequency as 45
    epochs = mne.make_fixed_length_epochs(data, duration=5, overlap = 1) #split the data into segments
    array = epochs.get_data() #this will convert the data to a numpy array
    return array



In [15]:
#let us read a data
sample_data = read_data(healthy_file_path[0])

Extracting EDF parameters from /Users/folasewaabdulsalam/Signal_Processing/Schizophrenia_Classification/schizophrenia_dataset/h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
231 matching events found
N

In [16]:
sample_data.shape #231 - no of epochs, 19 - no of channels, 1250, length of signal

(231, 19, 1250)

In [18]:
import sys
import os

# Suppress output
class SuppressOutput:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    
    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout.close()
        sys.stdout = self._original_stdout
# Wrap in SuppressOutput context manager
with SuppressOutput(): #mainly using %%capture could have worked if I were using a jupyter kernel, but since it is on vscode, I had to create a function called suppress outout and wrap my lines of code
    #let's read all the files, the use of capture is to not allow those information above to print for all the data
    healthy_epochs_array = [read_data(i) for i in healthy_file_path]
    schizo_epochs_array = [read_data(i) for i in schizo_patient_file_path]

In [19]:
healthy_epochs_array[0].shape

(231, 19, 1250)

In [ ]:
#combining both of the files